In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 
import xml.etree.ElementTree as ET
import cv2
import random


from functions import checkLabel
from functions import checkCoordinate
from functions import yolo_to_pascal
from xmlMaker import init_xml
from xmlMaker import add_object
from xmlMaker import prettyXml
from xmlMaker import bbox_resize

# find image set with label 0 in train and test set.

save result in test_0_list.txt and train_0_list.txt

In [2]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'
for name in tqdm(folderName):
    tmpeList = []
    labelsPath = os.path.join(datasetPath, '{}/labels/'.format(name))
    labelList = os.listdir(labelsPath)
    for label in tqdm(labelList):
        labelPath = os.path.join(labelsPath, label)
        if checkLabel(labelPath) is not None:
            tmpeList.append(checkLabel(labelPath))
    savePath = os.path.join(datasetPath, '{}_0_list.txt'.format(name))
    with open(savePath, 'w') as f:
        for image in tmpeList:
            f.write ('{}\n'.format(image))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4306 [00:00<?, ?it/s]

  0%|          | 0/17221 [00:00<?, ?it/s]

# check label

check wether the label is right  
there is no output if all labels are right

In [4]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'
for name in tqdm(folderName):
    labelsPath = os.path.join(datasetPath, '{}/labels/'.format(name))
    
    imagesPath = os.path.join(datasetPath, '{}/images/'.format(name))
    txtPath = os.path.join(datasetPath, '{}_0_list.txt'.format(name))
    with open(txtPath, 'r') as f:
        txtRead = [x.strip() for x in f.readlines()]

    for image in tqdm(txtRead):
        labelPath = os.path.join(labelsPath, '{}.txt'.format(image))
        
        imagePath = os.path.join(imagesPath, '{}.jpg'.format(image))
        imageArray = plt.imread(imagePath)
        imageWidth = imageArray.shape[1]
        imageHeight = imageArray.shape[0]
        with open(labelPath, 'r') as f:
            labelRead = [x.strip() for x in f.readlines()]

        for idx, line in enumerate(labelRead):
            label = int(line[0])
            if label == 0:
                bbox = line.split(' ')[1:]
                bbox = np.array(bbox, dtype = np.float64)
                # yolo -> pascal
                bbox = yolo_to_pascal(bbox, imageWidth, imageHeight)
                callBack = checkCoordinate(bbox, pascalFormat=True, 
                                          imageWidth=imageWidth, imageHeight=imageHeight)
                if callBack is not None:
                    print(labelPath)
                    print("wrong label index: {}".format(idx+1))
                    print("wrong label: {}".format(callBack))
                    print("----------------------------------------------------------------------")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2081 [00:00<?, ?it/s]

  0%|          | 0/8444 [00:00<?, ?it/s]

# mix txt

mix test_0_list.txt with test_1_list.txt  
mix train_0_list.txt with train_1_list.txt  

In [4]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'
test_list = []
train_list = []

for name in tqdm(folderName):
    for label in tqdm([0, 1]):
        txtName = '{}_{}_list.txt'.format(name, label)
        txtPath = os.path.join(datasetPath, txtName)
        with open(txtPath, 'r') as f:
            txtRead = [x.strip() for x in f.readlines()]

        for image in tqdm(txtRead):
            if image not in eval('{}_list'.format(name)):
                eval('{}_list'.format(name)).append(image)

    savePath = os.path.join(datasetPath, '{}_list.txt'.format(name))
    with open(savePath, 'w') as f:
        for image in eval('{}_list'.format(name)):
            f.write('{}\n'.format(image))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2081 [00:00<?, ?it/s]

  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8444 [00:00<?, ?it/s]

  0%|          | 0/4707 [00:00<?, ?it/s]

# make XML file

load image name in test_list.txt and train_list.txt

In [2]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'
xmlFolderPath = os.path.join(datasetPath, 'Annotations/')
if not os.path.exists(xmlFolderPath):
    os.mkdir(xmlFolderPath)

for name in tqdm(folderName):
    imagesPath = os.path.join(datasetPath, '{}/images/'.format(name))
    labelsPath = os.path.join(datasetPath, '{}/labels/'.format(name))
    imageListPath = os.path.join(datasetPath, '{}_list.txt'.format(name))
    with open(imageListPath, 'r') as f:
        imageList = [x.strip() for x in f.readlines()]

    for image in tqdm(imageList):
        imagePath = os.path.join(imagesPath, '{}.jpg'.format(image))
        imageArray = plt.imread(imagePath)
        imageWidth = imageArray.shape[1]
        imageHeight = imageArray.shape[0]
        xmlPath = os.path.join(xmlFolderPath, '{}.xml'.format(image))
        txtPath = os.path.join(labelsPath, '{}.txt'.format(image))
        with open(txtPath, 'r') as f:
            txtRead = [x.strip() for x in f.readlines()]
        # init xml
        if not os.path.exists(xmlPath):
            # image width, image height are unified
            init_xml(xmlPath, 'D-Fire', image, 'D-Fire_fire_detection', imageWidth=640, imageHeight=480)
        for object in txtRead:
            objectClass = int(object[0])
            yolobbox = [float(x) for x in object.split(' ')[1:]]
            yolobbox = np.array(yolobbox, dtype=np.float64)
            pascalbbox = yolo_to_pascal(yolobbox, imageWidth=imageWidth, imageHeight=imageHeight)
            widthRatio = float(640 / imageWidth)
            heightRatio = float(480 / imageHeight)
            bboxResized = bbox_resize(pascalbbox, widthRatio=widthRatio, heightRatio=heightRatio)
            # parse xml file
            tree = ET.parse(xmlPath)
            if objectClass == 0:
                add_object(tree, bboxResized, 'smoke')
                tree.write(xmlPath)
            else:
                add_object(tree, bboxResized, 'fire')
                tree.write(xmlPath)
    
        # reshape xml
        root = tree.getroot()
        prettyXml(root, '\t', '\n')
        # save
        tree.write(xmlPath)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2301 [00:00<?, ?it/s]

  0%|          | 0/9388 [00:00<?, ?it/s]

# generate ImageSets and JPEGImages floders

## make JPEGImages

copy images after reshaping into JPEGImages foldes

In [3]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'

jpegimagesPath = os.path.join(datasetPath, 'JPEGImages/')
# make JPEGImages folder
if not os.path.exists(jpegimagesPath):
    os.mkdir(jpegimagesPath)

for name in tqdm(folderName):
    imagesPath = os.path.join(datasetPath, '{}/images/'.format(name))
    imageListPath = os.path.join(datasetPath, '{}_list.txt'.format(name))
    with open(imageListPath, 'r') as f:
        imageList = [x.strip() for x in f.readlines()]
        
    for image in tqdm(imageList):
        savePath = os.path.join(jpegimagesPath, '{}.jpg'.format(image))
        if not os.path.exists(savePath):
            imagePath = os.path.join(imagesPath, '{}.jpg'.format(image))
            imageArray = cv2.imread(imagePath)
            imageResized = cv2.resize(imageArray, (640, 480))
            cv2.imwrite(savePath, imageResized)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2301 [00:00<?, ?it/s]

  0%|          | 0/9388 [00:00<?, ?it/s]

## make ImageSets

loading all image names, distribute them into trainset(70%) and testset(30%)

In [68]:
folderName = ['test', 'train']
datasetPath = '../D-Fire/'
ImageSetsPath = os.path.join(datasetPath, 'ImageSets/Main/')

allImage = []

for name in folderName:
    imageListPath = os.path.join(datasetPath, '{}_list.txt'.format(name))
    with open(imageListPath, 'r') as f:
        imageList = [x.strip() for x in f.readlines()]

    for image in imageList:
        allImage.append(image)

allImage = np.array(allImage)

testSize = int(len(allImage) * 0.3)
# shuffle
np.random.shuffle(allImage)
# sample
testIndex = random.sample(range(len(allImage)), testSize)
testList = allImage[np.array(testIndex)]
trainList = [item for item in allImage if item not in testList]
# save
trainTxtPath = os.path.join(ImageSetsPath, 'trainval.txt')
testTxtPath = os.path.join(ImageSetsPath, 'test.txt')

with open(trainTxtPath, 'w') as f:
    for image in trainList:
        f.write('{}\n'.format(image))

with open(testTxtPath, 'w') as f:
    for image in testList:
        f.write('{}\n'.format(image))